In [2]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import tiktoken
#from train_gpt2 import GPT, GPTConfig

In [3]:
import torch
import tiktoken

In [4]:
with open('input.txt', 'r') as f:
    text = f.read()


In [5]:
16384 * 10

163840

In [6]:
print("First Citizen:\nBefore we proceed any further, hear me speak.")

First Citizen:
Before we proceed any further, hear me speak.


In [7]:
x = torch.tensor([[2, 3, 34, 4, 5], [1, 2, 3, 4, 5]])
x.shape

torch.Size([2, 5])

In [8]:
x[1]

tensor([1, 2, 3, 4, 5])

In [9]:
class DataLoader:
    def __init__(self, data, B, T, process_rank, num_processes):
        self.B = B
        self.T = T
        self.process_rank = process_rank
        self.num_processes = num_processes
        tokenizer = tiktoken.get_encoding("gpt2")
        tokenized_data = tokenizer.encode(data)
        self.tok_data = torch.tensor(tokenized_data)
        self.starting = self.B * self.T * self.process_rank
        self.ending = self.starting + (self.B * self.T)
        print(f"Batches: {len(self.tok_data) // (self.B * self.T)}")
        print("Tokenized data length:", len(self.tok_data))

    def next_batch(self):
        batch_X = self.tok_data[self.starting:self.ending]
        batch_Y = self.tok_data[self.starting + 1: self.ending + 1]
        self.starting += self.B * self.T * self.num_processes
        self.ending += self.B * self.T * self.num_processes
        batch_X = batch_X.view(self.B, self.T).to("cuda")
        batch_Y = batch_Y.view(self.B, self.T).to("cuda")
        if self.ending >= len(self.tok_data):
            self.starting = 0
            self.ending = self.B * self.T
        return batch_X, batch_Y

    def num_of_batches(self):
        return len(self.tok_data) // (self.B * self.T)

test = DataLoader(text, 500, 64, 0, 0)
test.next_batch(), test.num_of_batches()

Batches: 10
Tokenized data length: 338024


((tensor([[ 5962, 22307,    25,  ...,   760,   327,  1872],
          [  385,  1526, 28599,  ...,  1309,   340,   307],
          [ 1760,    25,  1497,  ...,   198,  1929,  4316],
          ...,
          [  262,   198,   805,  ...,   198, 32454,    25],
          [  198,  5779,    11,  ...,    25,   788,   760],
          [  502,   407,    11,  ...,  3046,   312,  3754]], device='cuda:0'),
  tensor([[22307,    25,   198,  ...,   327,  1872,   385],
          [ 1526, 28599,   318,  ...,   340,   307,  1760],
          [   25,  1497,    11,  ...,  1929,  4316,   462],
          ...,
          [  198,   805,    11,  ..., 32454,    25,   198],
          [ 5779,    11,  1309,  ...,   788,   760,   502],
          [  407,    11,   198,  ...,   312,  3754,  7363]], device='cuda:0')),
 10)

In [10]:
test = DataLoader(text, 500, 64, 0, 1)
test.next_batch()

Batches: 10
Tokenized data length: 338024


(tensor([[ 5962, 22307,    25,  ...,   760,   327,  1872],
         [  385,  1526, 28599,  ...,  1309,   340,   307],
         [ 1760,    25,  1497,  ...,   198,  1929,  4316],
         ...,
         [  262,   198,   805,  ...,   198, 32454,    25],
         [  198,  5779,    11,  ...,    25,   788,   760],
         [  502,   407,    11,  ...,  3046,   312,  3754]], device='cuda:0'),
 tensor([[22307,    25,   198,  ...,   327,  1872,   385],
         [ 1526, 28599,   318,  ...,   340,   307,  1760],
         [   25,  1497,    11,  ...,  1929,  4316,   462],
         ...,
         [  198,   805,    11,  ..., 32454,    25,   198],
         [ 5779,    11,  1309,  ...,   788,   760,   502],
         [  407,    11,   198,  ...,   312,  3754,  7363]], device='cuda:0'))

In [11]:
test.next_batch()

(tensor([[ 7363,    25,   318,  ...,   345,    13,   198],
         [  198, 44879,    40,  ...,    11,   355,   705],
         [ 4246,   567,    11,  ...,    11,   416,   617],
         ...,
         [   35,    52,  3398,  ...,  4490,   321,    11],
         [  345, 13796,   326,  ...,    11,  4167,     0],
         [  262,  5822,   288,  ..., 18516,   620,   902]], device='cuda:0'),
 tensor([[   25,   318,   339,  ...,    13,   198,   198],
         [44879,    40,  3535,  ...,   355,   705,  4246],
         [  567,    11,   287,  ...,   416,   617,  2863],
         ...,
         [   52,  3398,  7597,  ...,   321,    11,   345],
         [13796,   326,   339,  ...,  4167,     0,   262],
         [ 5822,   288,   849,  ...,   620,   902,   284]], device='cuda:0'))

In [12]:
test2 = DataLoader(text, 1, 16, 0 ,1)

Batches: 21126
Tokenized data length: 338024


In [13]:
test2.next_batch()

(tensor([[ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
           3285,   502,  2740,    13,   198,   198]], device='cuda:0'),
 tensor([[22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,  3285,
            502,  2740,    13,   198,   198,  3237]], device='cuda:0'))

In [14]:
B, T = 16, 1024
total_batch_size = 524288
grad_accum_steps = total_batch_size // (B * T)
grad_accum_steps

32

In [15]:
1024 * 16

16384

In [16]:
100 // (320031//16384)

5

In [17]:
32*50 * (1024 * 16)

26214400

In [18]:
(int)(len(text) * 0.05)

55769

In [19]:
train = text[:(int)(len(text) * 0.95)]
val = text[(int)(len(text) * 0.95):]

In [20]:
len(train), len(val)

(1059623, 55770)

In [21]:
100 - 600000 / len(text)

99.46207300924428

In [22]:
32 * 256 * 2

16384

In [23]:
55770 / 128

435.703125

In [24]:
len(train)/len(text), len(val)/len(text)

(0.9499996862092553, 0.050000313790744606)

In [25]:
t = torch.tensor([1,2,3,4,5])
t[3].item()

4

In [26]:
class DataLoader:
    def __init__(self, data, B, T):
        self.B = B
        self.T = T
        tokenizer = tiktoken.get_encoding("gpt2")
        tokenized_data = tokenizer.encode(data)
        self.tok_data = torch.tensor(tokenized_data)
        self.starting = 0
        self.ending = self.B * self.T
        print(f"Batches: {len(self.tok_data) // (self.B * self.T)}")
    def next_batch(self):
        batch_X = self.tok_data[self.starting:self.ending]
        batch_Y = self.tok_data[self.starting + 1: self.ending + 1]
        self.starting += self.B * self.T
        self.ending += self.B * self.T
        batch_X = batch_X.view(self.B, self.T).to("cuda")
        batch_Y = batch_Y.view(self.B, self.T).to("cuda")
        if self.ending >= len(self.tok_data):
            self.starting = 0
            self.ending = self.B * self.T
        return batch_X, batch_Y

trainloader = DataLoader(text, 1, 32)

Batches: 10563


In [27]:
x, y = trainloader.next_batch()
x, y

(tensor([[ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
           3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
            461,    11,  2740,    13,   198,   198,  5962, 22307,    25,   198,
           1639,   389]], device='cuda:0'),
 tensor([[22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,  3285,
            502,  2740,    13,   198,   198,  3237,    25,   198,  5248,   461,
             11,  2740,    13,   198,   198,  5962, 22307,    25,   198,  1639,
            389,   477]], device='cuda:0'))

In [28]:
model = GPT(GPTConfig)

NameError: name 'GPT' is not defined

In [10]:
tokenizer = tiktoken.get_encoding("gpt2")

In [15]:
tokens = tokenizer.encode(text)

In [16]:
4 * 32

128

In [49]:
x = torch.tensor(tokens[:128]).view(4, 32)
x.shape, x.view(-1).shape

(torch.Size([4, 32]), torch.Size([128]))

In [64]:
tokens = torch.tensor(tokenizer.encode(text))

: 

In [ ]:
def get_batch(split, batch_size=32, block_size = 128):
    indxs = torch.randint(0, len(split) - block_size, (batch_size, ))
    x = torch.stack([split[i: i + block_size] for i in indxs])
    y = torch.stack([split[i+1: i + block_size + 1] for i in indxs])
    x, y = x.to('cuda'), y.to('cuda')
    return x, y

In [61]:
x, y = get_batch(tokens)
x[0], y[0]

(tensor([  287,  1745,    11,   198,  6653, 18887,  4957,    11,  4950, 20828,
         22260,    11,   198,  1870,   607, 34224,    82,   422,   502,   290,
           584,   517,    11,   198,    50,  5013,   669,   284,   607,   290,
         14987,   287,   616,  1842,    11,   198, 15979,  2752,   340,   257,
          1517,  5340,    11,   198,  1890,   883, 22448,   314,   423,   878,
         28779,   276,    11,   198,  2504,  1683, 18341,   283,  1437,   481,
           307, 36440,  1549,    26,   198, 26583,   428,  1502, 22027, 18226,
         12523, 20486,     6,   268,    11,   198,  2504,  4844,  2236,   423,
          1895, 12722, 41227,  6888,   198,    51,   359, 18341,   283,  1437,
           262,  1090,   301,   423,  1392,   257,  5229,    13,   198,   198,
         10761,  5883,  9399,    25,   198,    42,   776,   283,  1437,   262,
          1090,   301,     0,   198,    32,  3670,   329,   257, 25920,   286,
           477,  8714,   262,  5290,    13,   198,  